# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




#### Creating the table "item_in_specific_sessions" to insert session/event data

In [21]:
# Drop Table if Exist
delete_sessions_table = "DROP TABLE IF EXISTS item_in_specific_sessions"
session.execute(delete_sessions_table)

create_sessions_table = "CREATE TABLE IF NOT EXISTS item_in_specific_sessions \
                         (session_id int,\
                         item_in_session int, \
                         artist text,\
                         length float, \
                         song_title text, \
                         PRIMARY KEY (session_id, item_in_session))"
session.execute(create_sessions_table)

#### Insert  data into item_in_specific_sessions table

In [22]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## INSERT statements into the `query` variable
        insert_session = "INSERT INTO item_in_specific_sessions\
                        (\
                        session_id,\
                        item_in_session,\
                        artist,length,\
                        song_title)"
        query = insert_session + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]),int(line[3]), ascii(line[0]),  float(line[5]),  ascii(line[9])))

#### SELECT statement to verify that the data have been inserted into the table

In [26]:
select_session = "select artist, song_title, length from item_in_specific_sessions \
                  WHERE session_id = 338 and item_in_session = 4"
rows = session.execute(select_session)

In [25]:
from prettytable import PrettyTable
x = PrettyTable()

x.field_names = ["Artist", "Song Tittle", "Length"]
x.add_row([row.artist, row.song_title, row.length])
print(x)

+-------------+-----------------------------------+--------------------+
|    Artist   |            Song Tittle            |       Length       |
+-------------+-----------------------------------+--------------------+
| 'Faithless' | 'Music Matters (Mark Knight Dub)' | 495.30731201171875 |
+-------------+-----------------------------------+--------------------+


### Creating the table "songplay_specific_users" to insert songplay data

In [27]:
#Dropping the table songplay_specific_users if exist
delete_users_table = "DROP TABLE IF EXISTS songplay_specific_users"
session.execute(delete_users_table)

#Creating the table songplay_specific_users 
create_users_table = "CREATE TABLE IF NOT EXISTS songplay_specific_users \
                    (user_id int, \
                    session_id int, \
                    item_in_session int,\
                    artist text,\
                    first_name text, \
                    last_name text, \
                    song_title text, \
                    PRIMARY KEY ((user_id, session_id), item_in_session))"
session.execute(create_users_table)                    

### Insert data into  "songplay_specific_users" table

In [29]:
#Insert all data into the users table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO songplay_specific_users (\
                user_id,\
                session_id, \
                item_in_session, \
                artist, \
                first_name, \
                last_name, \
                song_title)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]),int(line[8]), int(line[3]),line[0], line[1], line[4], line[9])) 

#### SELECT statement to verify that the data have been inserted into the "songplay_specific_users" table correctly

In [79]:
select_query_spcuser = "select artist, \
                        song_title, \
                        first_name, \
                        last_name \
                        from songplay_specific_users\
                        WHERE session_id = 182 and user_id = 10"
rows = session.execute(select_query_spcuser)
for i in rows:
    print (i.artist, i.song_title, i.first_name, i.last_name)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### Creating table  "users_listen_specific_song" table

In [84]:
#Create the users_listen_specific_song tablefor query 3
drop_song_listens = "DROP TABLE IF EXISTS users_listen_specific_song"
session.execute(drop_song_listens)

create_song_listens = "CREATE TABLE IF NOT EXISTS users_listen_specific_song ( \
                      song_title text, \
                      user_id int, \
                      first_name text,\
                      last_name text, \
                      PRIMARY KEY (song_title, user_id))"
session.execute(create_song_listens)

### Insert data into  "users_listen_specific_song" table

In [86]:
#Insert all data into the users_listen_specific_song table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO users_listen_specific_song \
                (song_title, \
                user_id,\
                first_name, \
                last_name)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]),line[1], line[4])) 

#### SELECT statement to verify that the data have been inserted into the "users_listen_specific_song" table correctly

In [89]:
#Query for every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query3 = "select first_name, last_name from users_listen_specific_song WHERE song_title = 'All Hands Against His Own'"
rows = session.execute(query3)
for row in rows:
    print (row.first_name, row.last_name)
                   

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [90]:
# Drop Table item_in_specific_sessions
delete_sessions_table = "DROP TABLE IF EXISTS item_in_specific_sessions"
session.execute(delete_sessions_table)
#Dropping the table songplay_specific_users if exist for query2
delete_users_table = "DROP TABLE IF EXISTS songplay_specific_users"
session.execute(delete_users_table)
# Drop Table users_listen_specific_song
drop_song_listens = "DROP TABLE IF EXISTS users_listen_specific_song"
session.execute(drop_song_listens)

### Close the session and cluster connection¶

In [26]:
session.shutdown()
cluster.shutdown()